<a href="https://colab.research.google.com/github/Tolulopeelijah/Cats-vs-Dog-classifier/blob/main/cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tolulopeelijah","key":"210a7e92a996c2d01970b24cf756b16f"}'}

In [ ]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

100% 810M/812M [00:22<00:00, 37.8MB/s]
100% 812M/812M [00:22<00:00, 37.1MB/s]


In [ ]:
! unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [ ]:
 ! unzip train.zip
 ! unzip test1.zip

Archive:  train.zip
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os, shutil
import pandas as pd
import numpy as np
from PIL import Image
from random import sample
from math import ceil

In [ ]:
print(len(os.listdir('./train')))
print(len(os.listdir('./test1')))

In [ ]:
path = './data/'
subpaths = ['train/', 'test/']
subsub = ['cats', 'dogs']
for subpath in subpaths:
  for sub in subsub:
    dir = path+subpath+sub
    os.makedirs(dir, exist_ok = True)

In [ ]:
init_dest_path = './train/'
dest_path = './data/train/'
#  using 80% of the training set for the training and the other twenty for validation
for img in sample(os.listdir(init_dest_path), ceil(0.8 * len(os.listdir(init_dest_path)))):
  init_dest = init_dest_path + img
  image_class = 'dogs' if img[:3] == 'dog' else 'cats' if img[:3] == 'cat' else 'none'
  if image_class != 'none':
    fin_dest = dest_path + image_class
  shutil.move(init_dest, fin_dest)


for img in (os.listdir(init_dest_path)):
  init_dest = init_dest_path + img
  image_class = 'dogs' if img[:3] == 'dog' else 'cats' if img[:3] == 'cat' else 'none'
  if image_class != 'none':
    fin_dest = './data/test/' + image_class
  shutil.move(init_dest, fin_dest)

In [ ]:
print(f'length of cats in train: {len(os.listdir("./data/train/cats/"))}')
print(f'length of dogs in train: {len(os.listdir("./data/train/dogs"))}')
print(f'length of cats in validation: {len(os.listdir("./data/test/cats"))}')
print(f'length of dogs in validation: {len(os.listdir("./data/test/dogs"))}')

In [ ]:
10010 + 999 + 2490 + 2510

In [ ]:
path = './data/train/'
images = os.listdir('./data/train/cats')[:8] + os.listdir('./data/train/dogs')[:8]
plt.figure(figsize = (15,15))
for num, image_name in enumerate(images):
  image_path = path + 'cats/' + image_name if image_name[:3] == 'cat' else path + 'dogs/' + image_name
  img = Image.open(image_path)
  im = np.asarray(img)
  plt.axis('off')
  plt.subplot(4,4, num+1)
  plt.imshow(im)

In [ ]:
path = './test1/'
plt.figure(figsize = (15,15))
for num, image_name in enumerate(os.listdir(path)[:16]):
  image_path = path + image_name
  img = Image.open(image_path)
  im = np.asarray(img)
  plt.axis('off')
  plt.subplot(4,4, num+1)
  plt.imshow(im)

In [ ]:
model = keras.models.Sequential([keras.layers.Conv2D(32, (3,3), input_shape = (300, 300, 3), activation = 'relu'),
                                 keras.layers.MaxPool2D((2,2)),
                                 keras.layers.Conv2D(64, (3,3), activation = 'relu'),
                                 keras.layers.MaxPool2D((2,2)),
                                 keras.layers.Conv2D(64, (3,3), activation = 'relu'),
                                 keras.layers.MaxPool2D((2,2)),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(500, activation = 'relu'),
                                 keras.layers.Dense(1, activation = 'sigmoid')])

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.BinaryCrossentropy(), metrics = 'accuracy')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_generator = ImageDataGenerator(rescale = 1/255)
val_data_generator = ImageDataGenerator(rescale = 1/255)

train_data = train_data_generator.flow_from_directory('./data/train', target_size = (300,300), batch_size = 20, class_mode = 'binary')
val_data = val_data_generator.flow_from_directory('./data/test', target_size = (300,300), batch_size = 20, class_mode = 'binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
history = model.fit(train_data, 
                    steps_per_epoch = 1000,
                    epochs = 20,
                    verbose = 1,
                    validation_data = val_data,
                    validation_steps = 250)

Epoch 1/20
1000/1000 [==============================] - 107s 106ms/step - loss: 0.6648 - accuracy: 0.6007 - val_loss: 78.4746 - val_accuracy: 0.5925
Epoch 2/20
1000/1000 [==============================] - 104s 104ms/step - loss: 0.6061 - accuracy: 0.6783 - val_loss: 43.0510 - val_accuracy: 0.6290
Epoch 3/20
1000/1000 [==============================] - 103s 103ms/step - loss: 0.5487 - accuracy: 0.7235 - val_loss: 56.4937 - val_accuracy: 0.6310
Epoch 4/20
1000/1000 [==============================] - 100s 100ms/step - loss: 0.4249 - accuracy: 0.8036 - val_loss: 117.8515 - val_accuracy: 0.6190
Epoch 5/20
1000/1000 [==============================] - 100s 100ms/step - loss: 0.2096 - accuracy: 0.9137 - val_loss: 212.0852 - val_accuracy: 0.6040
Epoch 6/20
1000/1000 [==============================] - 100s 100ms/step - loss: 0.0619 - accuracy: 0.9789 - val_loss: 486.7854 - val_accuracy: 0.6110
Epoch 7/20
1000/1000 [==============================] - 100s 100ms/step - loss: 0.0375 - accuracy: 0.98